In [7]:
import pickle
import numpy as np
import pandas as pd
import polars as pl
import scanpy as sc

In [5]:
df = pl.read_parquet("/home/wzc26/work/vcc/nbglm/data/all_de_cache/real_de.parquet")

In [6]:
df

fold_change,p_value,fdr,log2_fold_change,abs_log2_fold_change,target,feature
f32,f32,f32,f32,f32,cat,cat
0.05,0.560741,0.858703,-4.321928,4.321928,"""CHMP3""","""SAMD11"""
1.066832,0.001344,0.010684,0.093333,0.093333,"""CHMP3""","""NOC2L"""
1.068095,0.326018,0.672394,0.09504,0.09504,"""CHMP3""","""KLHL17"""
1.096326,0.446153,0.781066,0.132677,0.132677,"""CHMP3""","""PLEKHN1"""
9.176692,0.009665,0.057069,3.197974,3.197974,"""CHMP3""","""PERM1"""
…,…,…,…,…,…,…
0.136395,0.004848,0.032185,-2.874134,2.874134,"""DNAJA3""","""MT-ND4L"""
0.526046,3.7883e-13,1.0724e-11,-0.92674,0.92674,"""DNAJA3""","""MT-ND4"""
0.05,0.023489,0.116149,-4.321928,4.321928,"""DNAJA3""","""MT-ND5"""


In [13]:
import polars as pl

alpha = 0.05  # 显著性阈值

genes_never_sig = (
    df.lazy()
      .group_by("feature")
      .agg([
          (pl.col("fdr") < alpha).any().alias("any_sig"),       # 是否在任一target下显著
          pl.n_unique("target").alias("n_targets"),              # 该基因被测试的target数
          pl.min("fdr").alias("min_fdr")                         # 观察到的最小FDR（越大越“不变”）
      ])
      .filter(pl.col("any_sig") == False)                        # 从未显著
      .select(["feature", "n_targets", "min_fdr"])
      .collect()
)


In [14]:
genes_never_sig

feature,n_targets,min_fdr
cat,u32,f32
"""WEE2""",150,0.074082
"""LDHAL6B""",150,0.111553
"""LRP5L""",150,0.127757
"""OOSP1""",150,0.077059
"""OR5H14""",150,0.082387
…,…,…
"""OR5V1""",150,0.107547
"""KRTAP9-7""",150,0.056904
"""SAR1A""",150,0.078741


In [8]:
adata = sc.read_h5ad("/home/wzc26/work/vcc/nbglm/data/adata_pp.h5ad")
adata

AnnData object with n_obs × n_vars = 214109 × 18080
    obs: 'target_gene', 'guide_id', 'batch', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'S_score', 'G2M_score', 'phase'
    var: 'gene_id', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'

In [15]:
adata.obs

,target_gene,guide_id,batch,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,S_score,G2M_score,phase
AAACAAGCAACCTTGTACTTTAGG-Flex_1_01,CHMP3,CHMP3_P1P2_A|CHMP3_P1P2_B,Flex_1_01,8869,8869,53551.0,9.0,0.016806,-0.755976,0.667151,G2M
AAACAAGCATTGCCGCACTTTAGG-Flex_1_01,AKT2,AKT2_P1P2_A|AKT2_P1P2_B,Flex_1_01,7018,7018,23827.0,15.0,0.062954,-0.586524,0.737338,G2M
AAACCAATCAATGTTCACTTTAGG-Flex_1_01,SHPRH,SHPRH_P1P2_A|SHPRH_P1P2_B,Flex_1_01,9482,9482,74923.0,27.0,0.036037,-0.384098,0.341869,G2M
AAACCAATCCCTCGCTACTTTAGG-Flex_1_01,TMSB4X,TMSB4X_P1_A|TMSB4X_P1_B,Flex_1_01,8187,8187,39549.0,38.0,0.096083,-0.585026,0.569591,G2M
AAACCAATCTAAATCCACTTTAGG-Flex_1_01,KLF10,KLF10_P2_A|KLF10_P2_B,Flex_1_01,7853,7853,37409.0,5.0,0.013366,0.678200,-0.510825,S
...,...,...,...,...,...,...,...,...,...,...,...
TTTGCTGAGTTTGAGAATTCGGTT-Flex_3_16,TFAM,TFAM_P1P2_A|TFAM_P1P2_B,Flex_3_16,9323,9323,79265.0,4.0,0.005046,-0.007435,0.078003,G2M
TTTGTGAGTAGTAGCAATTCGGTT-Flex_3_16,KDM1A,KDM1A_P1P2_A|KDM1A_P1P2_B,Flex_3_16,8982,8982,54664.0,57.0,0.104273,0.664890,-0.638263,S
TTTGTGAGTCCATCCTATTCGGTT-Flex_3_16,non-targeting,non-targeting_00020|non-targeting_01323,Flex_3_16,8877,8877,64190.0,27.0,0.042063,0.958578,-0.783510,S
TTTGTGAGTCCTGACAATTCGGTT-Flex_3_16,BIRC2,BIRC2_P1P2_A|BIRC2_P1P2_B,Flex_3_16,7257,7257,25524.0,7.0,0.027425,-0.824117,0.859633,G2M


In [9]:
adata.obs

,target_gene,guide_id,batch,n_genes,n_genes_by_counts,total_counts,total_counts_mt,pct_counts_mt,S_score,G2M_score,phase
AAACAAGCAACCTTGTACTTTAGG-Flex_1_01,CHMP3,CHMP3_P1P2_A|CHMP3_P1P2_B,Flex_1_01,8869,8869,53551.0,9.0,0.016806,-0.755976,0.667151,G2M
AAACAAGCATTGCCGCACTTTAGG-Flex_1_01,AKT2,AKT2_P1P2_A|AKT2_P1P2_B,Flex_1_01,7018,7018,23827.0,15.0,0.062954,-0.586524,0.737338,G2M
AAACCAATCAATGTTCACTTTAGG-Flex_1_01,SHPRH,SHPRH_P1P2_A|SHPRH_P1P2_B,Flex_1_01,9482,9482,74923.0,27.0,0.036037,-0.384098,0.341869,G2M
AAACCAATCCCTCGCTACTTTAGG-Flex_1_01,TMSB4X,TMSB4X_P1_A|TMSB4X_P1_B,Flex_1_01,8187,8187,39549.0,38.0,0.096083,-0.585026,0.569591,G2M
AAACCAATCTAAATCCACTTTAGG-Flex_1_01,KLF10,KLF10_P2_A|KLF10_P2_B,Flex_1_01,7853,7853,37409.0,5.0,0.013366,0.678200,-0.510825,S
...,...,...,...,...,...,...,...,...,...,...,...
TTTGCTGAGTTTGAGAATTCGGTT-Flex_3_16,TFAM,TFAM_P1P2_A|TFAM_P1P2_B,Flex_3_16,9323,9323,79265.0,4.0,0.005046,-0.007435,0.078003,G2M
TTTGTGAGTAGTAGCAATTCGGTT-Flex_3_16,KDM1A,KDM1A_P1P2_A|KDM1A_P1P2_B,Flex_3_16,8982,8982,54664.0,57.0,0.104273,0.664890,-0.638263,S
TTTGTGAGTCCATCCTATTCGGTT-Flex_3_16,non-targeting,non-targeting_00020|non-targeting_01323,Flex_3_16,8877,8877,64190.0,27.0,0.042063,0.958578,-0.783510,S
TTTGTGAGTCCTGACAATTCGGTT-Flex_3_16,BIRC2,BIRC2_P1P2_A|BIRC2_P1P2_B,Flex_3_16,7257,7257,25524.0,7.0,0.027425,-0.824117,0.859633,G2M
